In [1]:
import psycopg2
import config as creds
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np
from numpy.random import randn
import pandas as pd
from scipy import stats
from datetime import datetime

CONNECT TO DATABASE:

In [2]:
user=creds.PGUSER
password=creds.PGPASSWORD
host=creds.PGHOST
port=5432
database=creds.PGDATABASE

engine_str=f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_str)
conn = engine.raw_connection()
print('Connected')
cur = conn.cursor()
print('Cursor created')

Connected
Cursor created


EXTRACT DATASET:

In [3]:
query='''
SELECT b.kbtu
      ,b.REPORTEDBUILDINGGROSSFLOORAREA
      ,b.dcrealpropertyid
      ,b.ward
      ,b.yearbuilt
      ,b.primarypropertytype_selfselect
      ,b.elegas
      ,n.awnd
      ,n.cldd
      ,n.htdd
      ,n.snow
      ,n.tavg
      ,n.wdf2
      ,n.wdf5
      ,n.wsf2
      ,n.wsf5
      ,n.date 
FROM buildings_data b
LEFT OUTER join noaa_data n
ON b.REPORTINGYEAR = n.WEATHERYEAR
WHERE b.MONTH = n.MONTH
AND b.ELEGAS = 'E'
AND b.PRIMARYPROPERTYTYPE_SELFSELECT = '18'
AND b.REPORTINGYEAR BETWEEN 2018 AND 2019
AND b.YEARBUILT > 0
AND b.REPORTEDBUILDINGGROSSFLOORAREA > 50000;
'''

data=pd.read_sql(query,conn)
data.head()

,kbtu,reportedbuildinggrossfloorarea,dcrealpropertyid,ward,yearbuilt,primarypropertytype_selfselect,elegas,awnd,cldd,htdd,snow,tavg,wdf2,wdf5,wsf2,wsf5,date
0,1989025.4,229914.0,0744S_0814,6,1908,18,E,9.8,0.0,906.0,1.2,35.8,300.0,310.0,36.0,49.0,2018-01
1,2491340.1,229914.0,0744S_0807,3,1964,18,E,9.2,53.0,133.0,0.0,62.3,200.0,200.0,38.9,55.0,2019-04
2,1907137.4,229914.0,0744S_0814,6,1908,18,E,11.6,0.0,663.0,4.5,43.6,320.0,300.0,47.0,62.0,2018-03
3,2398806.7,229914.0,0744S_0807,3,1964,18,E,10.1,3.0,567.0,0.3,46.8,350.0,350.0,45.0,59.0,2019-03
4,1864726.2,229914.0,0744S_0814,6,1908,18,E,10.7,17.0,319.0,0.0,54.9,320.0,320.0,35.1,46.0,2018-04


In [4]:
data.isnull().values.any()

False

FORMAT COLUMNS:

In [5]:
#CONVERT 'Date' COLUMN TO datetime format

#data["reportingyear"] = data["reportingyear"].astype(str)
#data['month']=data['month'].apply(lambda x: '{0:0>2}'.format(x))
#data['date_time'] = data[['reportingyear', 'month']].agg('-'.join, axis=1)
#data['date_time'] = (data.date_time + "-01")
#data['date_time'] = datetime.strptime('date_time', "%Y-%m-%d")
data['datetime']=pd.to_datetime(data['date'])

In [6]:
data['primarypropertytype_selfselect'].dtype

dtype('O')

In [7]:
data['primarypropertytype_selfselect']=data['primarypropertytype_selfselect'].astype('int32')

In [8]:
data.set_index('datetime', inplace=True)
data.head()

,kbtu,reportedbuildinggrossfloorarea,dcrealpropertyid,ward,yearbuilt,primarypropertytype_selfselect,elegas,awnd,cldd,htdd,snow,tavg,wdf2,wdf5,wsf2,wsf5,date
datetime,,,,,,,,,,,,,,,,,
2018-01-01,1989025.4,229914.0,0744S_0814,6,1908,18,E,9.8,0.0,906.0,1.2,35.8,300.0,310.0,36.0,49.0,2018-01
2019-04-01,2491340.1,229914.0,0744S_0807,3,1964,18,E,9.2,53.0,133.0,0.0,62.3,200.0,200.0,38.9,55.0,2019-04
2018-03-01,1907137.4,229914.0,0744S_0814,6,1908,18,E,11.6,0.0,663.0,4.5,43.6,320.0,300.0,47.0,62.0,2018-03
2019-03-01,2398806.7,229914.0,0744S_0807,3,1964,18,E,10.1,3.0,567.0,0.3,46.8,350.0,350.0,45.0,59.0,2019-03
2018-04-01,1864726.2,229914.0,0744S_0814,6,1908,18,E,10.7,17.0,319.0,0.0,54.9,320.0,320.0,35.1,46.0,2018-04


In [9]:
data.shape

(24, 17)

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(25,15))
ftr = list(["kbtu", "reportedbuildinggrossfloorarea", "ward", "yearbuilt", "awnd", "cldd", "htdd", "snow"])
corrMatrix = data[ftr].corr()
sns.heatmap(corrMatrix, annot=True, fmt='.1f', linewidths=.5)

DEFINE FEATURES:

In [ ]:
TARGET = "kbtu"
COLS = ['reportedbuildinggrossfloorarea', 'ward', 'yearbuilt', 'awnd', 'cldd', 'htdd', 'snow', 'datetime']

def make_sklearn_data(df=data, target=TARGET, cols=COLS):
    df = df.reset_index()
    X, y = df[cols], df[target]
    return X, y

In [ ]:
features = ['reportedbuildinggrossfloorarea', 'ward', 'yearbuilt', 'awnd', 'cldd', 'htdd', 'snow']
X, y = make_sklearn_data(cols=features)

In [ ]:
#Rank2D
from yellowbrick.features import Rank2D

# Instantiate the visualizer with the Pearson ranking algorithm
visualizer = Rank2D(algorithm='pearson', features=features, size=(1080, 720))

visualizer.fit(X, y)
visualizer.transform(X)
visualizer.show()

In [ ]:
# Instantiate the visualizer with the Covariance algorithm
visualizer = Rank2D(algorithm='covariance', features=features, size=(1080, 720))

visualizer.fit(X, y)
visualizer.transform(X)
visualizer.show()

In [ ]:
#Feature Importances
import yellowbrick as yb
from sklearn.ensemble import RandomForestRegressor
from yellowbrick.features import RadViz
from yellowbrick.features import FeatureImportances


model = RandomForestRegressor(n_estimators=10)
viz = FeatureImportances(model, labels=features, size=(1080, 720))

viz.fit(X, y)
viz.show()

In [ ]:
#Feature Importances
import yellowbrick as yb
from sklearn.linear_model import Lasso
from yellowbrick.features import RadViz
from yellowbrick.features import FeatureImportances


model = Lasso()
viz = FeatureImportances(model, labels=features, size=(1080, 720))

viz.fit(X, y)
viz.show()

In [ ]:
#CYCLIC ENCODER:  to capture temporal cycles (yearly).
from sklearn.base import BaseEstimator, TransformerMixin


class CyclicEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, date_extract="month"):
        if date_extract not in {"minute", "hour", "week", "month", "year"}:
            raise ValueError(f"specify correct date component to extract, not {date_extract}")
        self.date_extract = date_extract
    
    def get_date_component(self, x):
        if self.date_extract == "month":
            return x.dt.month
        elif self.date_extract == "year":
            return x.dt.year
        else:
            raise NotImplementedError(f"{self.date_extract} date component not implemented yet")
            
    def fit(self, X, y=None):
        self.cycle_max_ = self.get_date_component(X).max()
        return self

    def transform(self, X,  y=None):
        cols = []
        names = []
        x = self.get_date_component(X)
        xn = 2 * np.pi * x / self.cycle_max_
        cols.append(np.cos(xn))
        names.append(f"{X.name}_cos")
        cols.append(np.sin(xn))
        names.append(f"{X.name}_sin")
        return pd.DataFrame(np.asarray(cols).T, columns=names)

    
ce = CyclicEncoder().fit_transform(data.reset_index()["datetime"])
ce.plot(x="datetime_cos", y="datetime_sin", kind="scatter")

In [ ]:
#FEATURE EXTRACTION
from sklearn.base import clone
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder

extraction = Pipeline([
    ('column_selection', ColumnTransformer([
        ('time_components', FeatureUnion([
            ('month', CyclicEncoder(date_extract='month')), 
            ('year', CyclicEncoder(date_extract='year')), 
        ]), 'datetime'),
        ('ward_one_hot', OneHotEncoder(handle_unknown='ignore'), ['ward']),
        ('yearbuilt_one_hot', OneHotEncoder(handle_unknown='ignore'), ['yearbuilt']),
    ], remainder="passthrough")),
])

def make_energy_pipeline(model, append_transformers=None, fe=extraction):
    pipe = clone(fe)
    
    if append_transformers:
        for step in append_transformers:
            pipe.steps.append(step)
    
    pipe.steps.append(["model", clone(model)])
    return pipe

In [ ]:
#Test the Feature Extraction Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(*make_sklearn_data(), test_size=0.2)

model = make_energy_pipeline(LinearRegression())
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
#TIME SERIES CROSS VALIDATION
from functools import partial
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

rmse = partial(mean_squared_error, squared=False)


def time_series_evaluate(model, X, y):
    """
    Performs time series cross validation on the model, returning the
    cross validated r2, mse, and mae of the regressor, along with the 
    final fitted model, fitted on all of the data.
    """
    cv = TimeSeriesSplit(12)
    scores = {}
    
    scores["r2"] = cross_val_score(model, X, y, cv=cv, scoring=make_scorer(r2_score))
    scores["mse"] = cross_val_score(model, X, y, cv=cv, scoring=make_scorer(mean_squared_error))
#     scores["rmse"] = cross_val_score(model, X, y, cv=cv, scoring=make_scorer(rmse))
    scores["mae"] = cross_val_score(model, X, y, cv=cv, scoring=make_scorer(mean_absolute_error))
    
    model.fit(X, y)
    return model, scores

In [ ]:
# LINEAR MODEL
X, y = make_sklearn_data()
lm = make_energy_pipeline(LinearRegression())
time_series_evaluate(lm, X, y)

In [ ]:
#Second order polynomial regression
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import PolynomialFeatures

qm = make_energy_pipeline(SGDRegressor(), [('quad', PolynomialFeatures(2))])
time_series_evaluate(qm, X, y)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfm = make_energy_pipeline(RandomForestRegressor(n_estimators=10, max_depth=3))
time_series_evaluate(rfm, X, y)

In [ ]:
import time
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

start  = time.time()

r1 = LinearRegression()
r2 = RandomForestRegressor(n_estimators=10, random_state=1)

X, y = make_sklearn_data()
er = make_energy_pipeline(VotingRegressor([('lr', r1), ('rf', r2)]))
print(time_series_evaluate(er, X, y))
print("Time = {:0.3f} seconds".format(time.time()-start))

In [ ]:
conn.close()
print('Closed')